In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
173,AFG,Asia,Afghanistan,2020-07-01,31517.0,279.0,746.0,13.0,809.616,7.167,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
288,ALB,Europe,Albania,2020-07-01,2535.0,69.0,62.0,4.0,880.881,23.977,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
467,DZA,Africa,Algeria,2020-07-01,13907.0,336.0,912.0,7.0,317.142,7.662,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-30,France,164474.0,0.0,29822.0,0.0,0.0,0.0
0,2020-06-30,China,84836.0,0.0,4651.0,0.0,0.0,0.0
0,2020-06-30,Italy,240751.0,0.0,34790.0,0.0,0.0,0.0
0,2020-06-30,Spain,249338.0,0.0,28364.0,0.0,0.0,0.0
0,2020-06-30,United States,2628513.0,0.0,126652.0,0.0,0.0,0.0
0,2020-06-30,World,10481929.0,0.0,509631.0,0.0,0.0,0.0
0,2020-06-30,United Kingdom,313261.0,0.0,43677.0,0.0,0.0,0.0
0,2020-06-30,Germany,194612.0,0.0,8987.0,0.0,0.0,0.0
0,2020-06-30,Iran,228236.0,0.0,10801.0,0.0,0.0,0.0
0,2020-06-30,Turkey,199869.0,0.0,5130.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-01,France,164474.0,164801.0,29822.0,29843.0,-327.0,-21.0
0,2020-07-01,China,84836.0,84785.0,4651.0,4641.0,51.0,10.0
0,2020-07-01,Italy,240751.0,240578.0,34790.0,34767.0,173.0,23.0
0,2020-06-30,Spain,249338.0,249271.0,28364.0,28355.0,67.0,9.0
0,2020-07-01,United States,2628513.0,2634432.0,126652.0,127410.0,-5919.0,-758.0
0,2020-07-01,World,10481929.0,10446353.0,509631.0,511037.0,35576.0,-1406.0
0,2020-07-01,United Kingdom,313261.0,312654.0,43677.0,43730.0,607.0,-53.0
0,2020-07-01,Germany,194612.0,194725.0,8987.0,8985.0,-113.0,2.0
0,2020-07-01,Iran,228236.0,227662.0,10801.0,10817.0,574.0,-16.0
0,2020-07-01,Turkey,199869.0,199906.0,5130.0,5131.0,-37.0,-1.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")